# Overview

Pneumonia is an infection in one or both lungs. The infection causes inflammation in the air sacs in your lungs, which are called **alveoli**. The alveoli fill with fluid or pus, making it difficult to breathe.

![Pneumonia Inflammation](https://www.physio-pedia.com/images/9/94/Pneumonia_Inflammation.jpg)



Doctors or radiologiests conduct a physical exam and use **CXR**(chest x-ray) to examin and detect pneumonia. In CXR it shows opacity in the reagion.

![Pneumonia CXRs](https://ars.els-cdn.com/content/image/1-s2.0-S0092867418301545-figs6_lrg.jpg)


* There are multiple causes of opacity n CXR other than pneumonia like;
    * fluid overload (pulmonary edema)
    * bleeding
    * volume loss (atelectasis or collapse)
    * lung cancer
    * post-radiation or surgical changes
    * Outside of the lungs, fluid in the pleural space (pleural effusion) 

A number of factors such as positioning of the patient and depth of inspiration can alter the appearance of the CXR.



Pneumonia opacity can occour in different reagions of chest and the opacity can be of different kinds. This makes it the problem of detection(regression) as well as recognition(classification). For such purpose we can think of using the pre-existing models like 'Faster R-CNN' or 'Yolo'.

![Types and Regions](http://adigaskell.org/wp-content/uploads/2017/11/pneumonia-xray.jpg)




## Table of Contents

**- MileStone 1**
- Pre-Processing,-Data-Visualisation,-EDA-and-Model-Building
- Exploring-the-given-Data-files,-classes-and-images-of-different-classes
- Imports
- Load-images
- Load-Labels
- Load-Classes
- Show-bounding-box-on-the-image-to-identify-pneumonia
- Dealing-with-missing-values
- Get-Target-Labels-and-Classes-into-one-dataset
- Visualisation-of-different-classes
- Show-image-of-class-'No-Lung-Opacity-/-Not-Normal
- Show-image-of-class-'Normal'
- Show-image-of-class-'Lung-Opacity'
- Show-images-with-multiple-bounding-boxes-as-applicable
- Analysis-from-the-visualisation-of-different-classes
 
**- MileStone 2** 
- Build-Model
       

   

# Milestone 1: Pre-Processing, Data Visualisation, EDA and Model Building

## Exploring the given Data files, classes and images of different classes.

### Imports
[Back to top](#Table-of-Contents)

In [ ]:
import glob, pylab, pandas as pd
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import seaborn as sns
import gc
import glob
import os
import cv2
import pydicom

import warnings
warnings.simplefilter(action = 'ignore')

> ### Load Files


In [ ]:
detailed_df = pd.read_csv('/kaggle/input/rsna-pneumonia-detection-challenge/stage_2_detailed_class_info.csv')
train_df = pd.read_csv('/kaggle/input/rsna-pneumonia-detection-challenge/stage_2_train_labels.csv')

ROOT_DIR = '/kaggle/working'

In [ ]:

detailed_df.shape

In [ ]:

train_df.shape

In [ ]:
detailed_df.head()


In [ ]:
train_df.head()


Merging the data tables detailed_df and train_df¶


In [ ]:
df = pd.concat([train_df,detailed_df["class"]],axis=1,sort=False)
df = df.drop_duplicates()
df.info()

Summary on the values, types and null values:¶


In [ ]:
df.shape


In [ ]:
df.isnull().sum()

Distribution of classes

The following output shows that the nearly 2/3 of the patients do not have pneumonia (with target value = 0) and 1/3 of the patients have pneumonia (with target value =1

In [ ]:
pd.pivot_table(df,index=["Target"], values=['patientId'], aggfunc='count')

Distribution of patients in each class
There are 9555 patients in the category 'Lung Opacity' and 11821 in 'No Lung Opacity / Not Normal' category and 8851 are in Normal category

In [ ]:
pd.pivot_table(df,index=["class"], values=['patientId'], aggfunc='count')

The classes "No Lung Opacity / Not Normal", "Normal", and "Lung Opacity" are in the proportion of 39%, 29% and 32% respectively

In [ ]:
df["class"].value_counts().plot(kind='pie',autopct='%1.0f%%', shadow=True, subplots=False)

It is also clear from the below output that the patients who do not have pnuemonia do not have the bounding box coordinates

In [ ]:
pd.pivot_table(df,index=["Target"], aggfunc='count')

Count of patients having single row and more than single rows¶


In [ ]:
df['patientId'].value_counts().value_counts()

Patients who do not have pneumonia has only one record in the table¶


In [ ]:
df[df['Target'] == 0]['patientId'].value_counts().value_counts()

In [ ]:
sns.countplot(x = 'class', hue = 'Target', data = df)

Preprocessing - Filling the null values


In [ ]:
df.fillna(0.0)


Correlation between the variables
There is a strong Correlation between height and width variables

In [ ]:
df.corr()


In [ ]:
sns.jointplot(x = 'width', y = 'height', data = df, kind="reg")


## EDA with the header values from the dataframe
* Creating a data frame with all of their appropriate header values from the dicom file takes long time as there are 30277 records. Hence, the EDA analysis is done on a subset of randomly chosen 1000 records by keeping the same proportion of the classes. (i.e) The classes Not Normal, Normal, Lunge Opacity are in a proportion 39%, 29%, and 32% respectively.

* Number of rows of Not Normal class = 39% of 1000 = 390 rows
* Number of rows of Normal class = 29% of 1000 = 290 rows
* Number of rows of Lunge Opacity class = 32% of 1000 = 320 rows

In [ ]:
df_Not_Normal = df[df['class']=='No Lung Opacity / Not Normal'].sample(n=390)
df_Normal = df[df['class']=='Normal'].sample(n=290)
df_Lunge_Opacity = df[df['class']=='Lung Opacity'].sample(n=320)
frames = [df_Not_Normal, df_Normal, df_Lunge_Opacity]

dicom_df = pd.concat(frames)

dicom_df.shape

In [ ]:
def process_dicom_data(data_df):
    for n, pid in enumerate(data_df['patientId'].unique()):        
        dcm_file = '/kaggle/input/rsna-pneumonia-detection-challenge/stage_2_train_images/%s.dcm' % pid
        dcm_data = pydicom.read_file(dcm_file)        
        idx = (data_df['patientId']==dcm_data.PatientID)
        data_df.loc[idx,'Modality'] = dcm_data.Modality
        data_df.loc[idx,'PatientAge'] = pd.to_numeric(dcm_data.PatientAge)
        data_df.loc[idx,'PatientSex'] = dcm_data.PatientSex
        data_df.loc[idx,'BodyPartExamined'] = dcm_data.BodyPartExamined
        data_df.loc[idx,'ViewPosition'] = dcm_data.ViewPosition
        
    return data_df

In [ ]:
dicom_df = process_dicom_data(dicom_df)


In [ ]:
dicom_df = dicom_df.astype({"PatientAge": int})
dicom_df.fillna(0.0, inplace=True)
dicom_df.head()

> There are 995 unique patient rows exist
> 

In [ ]:
dicom_df.nunique()


## Now Visualizing the data along with their dicom header values


Patient's age proportion in the detection


In [ ]:
plt.figure(figsize = (30, 10))
sns.countplot(x = 'PatientAge', hue = 'Target', data = dicom_df)

Patient's gender proportion in the detection


In [ ]:
sns.countplot(x = 'PatientSex', hue = 'Target', data = dicom_df)


With respect to view proportion


In [ ]:
sns.countplot(x = 'ViewPosition', hue = 'Target', data = dicom_df);


In [ ]:
dicom_df = dicom_df.drop('Target', axis=1)

In [ ]:
dicom_df['PatientSex'].astype('category')
dicom_df['ViewPosition'].astype('category')
dicom_df['PatientSex'] = np.where(dicom_df["PatientSex"].str.contains("M"), 1, 0)
dicom_df['ViewPosition'] = np.where(dicom_df["ViewPosition"].str.contains("AP"), 1, 0)

In [ ]:
dicom_df.head()


Apart from the correlation between the width and height,there is no strong correlation between the other variables in the dataframe

In [ ]:
dicom_df.corr()


**Visualizing the dicom images**


In [ ]:
def show_dicom_image(data_df):
        img_data = list(data_df.T.to_dict().values())
        f, ax = plt.subplots(2,2, figsize=(16,18))
        for i,data_row in enumerate(img_data):
            pid = data_row['patientId']
            dcm_file = '/kaggle/input/rsna-pneumonia-detection-challenge/stage_2_train_images/%s.dcm' % pid
            dcm_data = pydicom.read_file(dcm_file)                    
            ax[i//2, i%2].imshow(dcm_data.pixel_array, cmap=plt.cm.bone)
            ax[i//2, i%2].set_title('ID: {}\n Age: {} Sex: {}'.format(
                data_row['patientId'],dcm_data.PatientAge, dcm_data.PatientSex))


In [ ]:
show_dicom_image(df[df['Target']==1].sample(n=4))


> Showing some random dicom images of a patient who do not have Pnuemonia, however with class No Lung Opacity / Not Normal

In [ ]:
show_dicom_image(df[ (df['Target']==0) & (df['class']=='No Lung Opacity / Not Normal')].sample(n=4))

> Showing some random dicom images of a patients who do not have Pnuemonia, however with class Normal

In [ ]:
show_dicom_image(df[ (df['Target']==0) & (df['class']=='Normal')].sample(n=4))


In [ ]:
def show_dicome_with_boundingbox(data_df):
    img_data = list(data_df.T.to_dict().values())
    f, ax = plt.subplots(2,2, figsize=(16,18))
    for i,data_row in enumerate(img_data):
        pid = data_row['patientId']
        dcm_file = '/kaggle/input/rsna-pneumonia-detection-challenge/stage_2_train_images/%s.dcm' % pid
        dcm_data = pydicom.read_file(dcm_file)                    
        ax[i//2, i%2].imshow(dcm_data.pixel_array, cmap=plt.cm.bone)
        ax[i//2, i%2].set_title('ID: {}\n Age: {} Sex: {}'.format(
                data_row['patientId'],dcm_data.PatientAge, dcm_data.PatientSex))
        rows = data_df[data_df['patientId']==data_row['patientId']]
        box_data = list(rows.T.to_dict().values())        
        for j, row in enumerate(box_data):            
            x,y,width,height = row['x'], row['y'],row['width'],row['height']
            rectangle = Rectangle(xy=(x,y),width=width, height=height, color="red",alpha = 0.1)
            ax[i//2, i%2].add_patch(rectangle)    

In [ ]:
show_dicome_with_boundingbox(df[df['Target']==1].sample(n=4))


# MileStone 2
# **Model Building**

In [ ]:
import keras
from keras.models import Sequential,Input,Model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import LeakyReLU
from keras.utils import to_categorical
import os

In [ ]:
images_list = os.listdir("/kaggle/input/rsna-pneumonia-detection-challenge/stage_2_train_images")
test_image = os.listdir("/kaggle/input/rsna-pneumonia-detection-challenge/stage_2_test_images")

In [ ]:
print(images_list[0])

In [ ]:
ty=[]

In [ ]:
c=df[df['patientId']==images_list[1][:-4]]

In [ ]:
ty.append(c['class'].unique()[0])

In [ ]:
path="/kaggle/input/rsna-pneumonia-detection-challenge/stage_2_train_images/"

In [ ]:
train_x=[]
train_y=[]
def makeDataset():
    for i in range(len(images_list)%1500):
        d=pydicom.read_file(path+images_list[i])
        c=df[df['patientId']==images_list[i][:-4]]
        train_y.append(c['class'].unique()[0])
        train_x.append(d.pixel_array)
makeDataset()

Defining labels

In [ ]:
def label(s):
    if s=='Normal':
        return 0
    if s=='No Lung Opacity / Not Normal':
        return 1
    if s=='Lung Opacity':
        return 2

Defining training parameters

In [ ]:
train_y = list(map(label,train_y))
train_y= np.stack(train_y)
train_x = np.stack(train_x)
batch_size = 2
epochs = 5
num_classes = 3

In [ ]:
train_y = to_categorical(train_y)
train_x=train_x.reshape(-1,1024,1024,1)

In [ ]:
train_x.shape

In [ ]:
train_x = train_x.astype('float32')
train_x = train_x / 255

In [ ]:
model= Sequential()
model.add(Conv2D(32,kernel_size=(7,7),activation='linear',input_shape=(1024,1024,1),padding='same'))
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPooling2D(pool_size=(7,7),padding='same'))
model.add(Conv2D(64,kernel_size=(7,7),activation='linear',padding='same'))
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPooling2D(pool_size=(7,7),padding='same'))
model.add(Conv2D(128,kernel_size=(7,7),activation='linear',padding='same'))
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPooling2D(pool_size=(7,7),padding='same'))
model.add(Flatten())
model.add(Dense(128,activation='linear'))
model.add(LeakyReLU(alpha=0.1))
model.add(Dense(num_classes,activation='softmax'))

In [ ]:
model.compile(loss=keras.losses.categorical_crossentropy, optimizer=keras.optimizers.Adam(),metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
model_train = model.fit(train_x,train_y,batch_size=batch_size,epochs=epochs,verbose=1)

In [ ]:
test_eval = model.evaluate(train_x, train_y, verbose=0)

In [ ]:
print('Test loss:', test_eval[0])
print('Test accuracy:', test_eval[1])